## Week 3 Part 1

In [ ]:
import pandas as pd
import numpy as np
import requests

In [ ]:
url="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

In [ ]:
page = requests.get(url)

In [ ]:
pageHTML = pd.read_html(page.content, header = 0)

In [ ]:
len(pageHTML)

3

In [ ]:
pageHTML[0]

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
...,...,...,...
175,M5Z,Not assigned,Not assigned
176,M6Z,Not assigned,Not assigned
177,M7Z,Not assigned,Not assigned
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


In [ ]:
type(pageHTML)

list

In [ ]:
type(pageHTML[0])

pandas.core.frame.DataFrame

Need to remove boroughs where it is Not Assigned

In [ ]:
df = pageHTML[0]

In [ ]:
df.shape

(180, 3)

In [ ]:
df = df[df["Borough"] != "Not assigned"]

In [ ]:
df.shape

(103, 3)

In [ ]:
df.reset_index(drop=True, inplace = True)

In [ ]:
df.columns

Index(['Postal Code', 'Borough', 'Neighbourhood'], dtype='object')

In [ ]:
df.index

RangeIndex(start=0, stop=103, step=1)

In [ ]:
df.index.name

In [ ]:
df.head()

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


## Week 3 part 2

Using the csv file to load the latitude and longitude values

In [ ]:
geourl="https://cocl.us/Geospatial_data"

In [ ]:
dfgeo = pd.read_csv(geourl)

In [ ]:
dfgeo.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Set the index for this dataframe

In [ ]:
dfgeo.set_index("Postal Code", drop=True, inplace=True)

In [ ]:
dfgeo.head()

,Latitude,Longitude
Postal Code,,
M1B,43.806686,-79.194353
M1C,43.784535,-79.160497
M1E,43.763573,-79.188711
M1G,43.770992,-79.216917
M1H,43.773136,-79.239476


Joing this dataframe to the one in part 1

In [ ]:
dfall = df.join(dfgeo, on="Postal Code")

In [ ]:
dfall.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [ ]:
dfall.shape

(103, 5)

## Week 3 part 3

Lets create a map with only certain boroughs

In [ ]:
import folium

In [ ]:
dfall[dfall["Borough"].str.contains("Toronto")].head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
19,M4E,East Toronto,The Beaches,43.676357,-79.293031


In [ ]:
mapToronto = folium.Map(location=[43.65, -79.36], zoom_start=12, width=850, height=700)

Loop through the dataframe and add markers

In [ ]:
dfTor = dfall[dfall["Borough"].str.contains("Toronto")]

In [ ]:
dfTor.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
19,M4E,East Toronto,The Beaches,43.676357,-79.293031


In [ ]:
for lat, long, borough, postcode in zip(dfTor["Latitude"], dfTor["Longitude"], dfTor["Borough"], dfTor["Postal Code"]):
  # print(borough, postcode)
  popupLabel = folium.Popup(f"{postcode}", parse_html=True)
  folium.CircleMarker(
      [lat, long],
      radius=2,
      popup=popupLabel,
      color="green"
  ).add_to(mapToronto)


In [ ]:
mapToronto